In [24]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression


In [ ]:
def get_data_paths():
    path = Path()

    saved_file_generator = path.joinpath("data_workaround").rglob("**/part-*")
    paths = [str(path.absolute()) for path in saved_file_generator]
    
    return paths
get_data_paths()  # nothing is hard coded, so don't worry about C:\\Lunky...

def turn_data_into_df():
    """
    When you run letsgo-win.bat, Seppe have setup variables "spark" and "sc".
    Variable "spark" is of the type spark: pyspark.sql.session.SparkSession
    
    Returns an object of the type spark DataFrame
    """
    path = Path()

    saved_file_generator = path.joinpath("data_workaround").rglob("**/part-*")
    paths = [str(path.absolute()) for path in saved_file_generator]
    
    return spark.read.json(paths)

In [2]:
data = spark.read.json(get_data_paths())
# data = spark.read.json("./data.json")
# use the second function if just reading ready data

NameError: name 'get_data_paths' is not defined

In [ ]:
df = turn_data_into_df()
df.show()

In [26]:
print(data.head())

Row(app_id='1774580', label=0, review_id='137421224', review_text="I'm really loving this game and want to love it even more but the horrible performance on my top end 4080 rig is unacceptable. I'm struggling to get 60fps even without ray tracing and i'd say wait a few weeks to pick this one up. hopefully it'll have a few updates in it by then and everyone can enjoy this great game.")


In [27]:
(train, test, val) = data.randomSplit([0.7, 0.15,0.15])

In [28]:
# first model - tokenizer -> TFHashing -> IDF -> LogReg
tokenizer = Tokenizer(inputCol="review_text", outputCol="words")
hashtf = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
lr = LogisticRegression(maxIter=100)

pipeline1 = Pipeline(stages=[tokenizer, hashtf, idf, lr]).fit(train)
predictions1 = pipeline1.transform(val)

In [29]:
# second model - tokenizer -> TFHashing -
tokenizer = Tokenizer(inputCol="review_text", outputCol="words")
cv = CountVectorizer(vocabSize=2**16, inputCol="words", outputCol='cv')
idf = IDF(inputCol='cv', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
lr = LogisticRegression(maxIter=100)
pipeline2 = Pipeline(stages=[tokenizer, cv, idf, lr]).fit(train)
predictions2 = pipeline2.transform(val)

In [30]:
accuracy1 = predictions1.filter(predictions1.label == predictions1.prediction).count() / float(val.count())
print("Accuracy Score 1: {0:.4f}".format(accuracy1))


Accuracy Score 1: 0.7817


In [31]:
accuracy2 = predictions2.filter(predictions2.label == predictions2.prediction).count() / float(val.count())
print("Accuracy Score 2: {0:.4f}".format(accuracy2))


Accuracy Score 2: 0.7676


In [32]:
# pipeline1.write().overwrite().save("./lr1.model")
# pipeline2.write().overwrite().save("./lr2.model")
pipeline2.save("./lr2.model")

In [6]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [22]:
# test of loading 
saved1 = PipelineModel.load("./lr1.model")
saved2 = PipelineModel.load("./lr2.model")

In [15]:
predictions3 = saved1.transform(val)
predictions4 

In [21]:
accuracy3 = predictions3.filter(predictions3.label == predictions3.prediction).count() / float(val.count())
print("Accuracy Score 3: {0:.4f}".format(accuracy3))


Accuracy Score 3: 0.7940
